## Tarea 002 - Análisis Multimodal MIR
### Gustavo Hernández Angeles

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from my_fun import *  # noqa: F403
from scipy.io import wavfile
from typing import Tuple
from pathlib import Path

## Problema 3

### Inciso a)

In [2]:
def f(a:float,
      delta:float) -> float:
    return np.sign(a) * delta * np.floor(np.abs(a)/delta + 0.5)

### Inciso b)

In [3]:
def plot_cuantizada_y_error(x: np.ndarray, 
                    y: np.ndarray,
                    delta: float,
                    graph_path: str,
                    figsize=(8, 2.2)):
    y_cuantizada = f(y, delta)
    plt.figure(figsize=figsize)
    plt.plot(x, y, label='Señal original')
    plt.step(x, y_cuantizada, label='Señal cuantizada', where='mid')
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Amplitud')
    plt.legend(loc='upper right')
    #plt.grid()
    plt.savefig(graph_path, bbox_inches='tight')
    plt.clf()
    
    plt.figure(figsize=figsize)
    plt.plot(x, np.abs(y - y_cuantizada), label='Error de cuantización')
    plt.xlabel('Tiempo [s]')
    plt.ylabel('Error de cuantización')
    plt.legend(loc='upper right')
    #plt.grid()
    plt.savefig(graph_path.replace('.pdf', '_error.pdf'), bbox_inches='tight')
    plt.clf()

In [4]:
## Función lineal
x = np.linspace(0, 5, 100)
y = 2 * x + 1
plot_cuantizada_y_error(x, y, delta=1, graph_path='../documento/figure/lineal_cuantizada.pdf')

<Figure size 800x220 with 0 Axes>

<Figure size 800x220 with 0 Axes>

In [5]:
## Función sinusoidal
x = np.linspace(0, 2 * np.pi, 100)
y = np.sin(x)
plot_cuantizada_y_error(x, y, delta=0.5, graph_path='../documento/figure/sinusoidal_cuantizada.pdf')

<Figure size 800x220 with 0 Axes>

<Figure size 800x220 with 0 Axes>

### Inciso c)

In [6]:
def obtener_delta(bits: int,
                  y: np.ndarray) -> np.float64:
    lmbda = 2 ** bits
    y_max = np.max(y).astype(np.float64)
    y_min = np.min(y).astype(np.float64)
    return np.abs(y_max - y_min) / (lmbda - 1)

def obtener_cuantizada(wav_path: Path,
                       bits: int) -> Tuple[int, np.ndarray]:
    audio = wavfile.read(wav_path)
    sample_rate = audio[0]
    data = audio[1]
    delta = obtener_delta(bits, data)
    data_cuantizada = f(data, delta)
    return sample_rate, data_cuantizada, data

In [7]:
bits = [2,4,6,8]
wav_path = Path("../data/audio_raw/piano-loops-127-octave-120-bpm.wav")

for b in bits:    
    sample_rate, data_cuantizada, _ = obtener_cuantizada(wav_path, b)
    wavfile.write(f"../data/audio_cuantizado/piano-loops-127-octave-120-bpm_{b}bits.wav",
                  sample_rate,
                  data_cuantizada.astype(np.int16))

/tmp/ipykernel_10380/1406793491.py:10: WavFileWarning: Chunk (non-data) not understood, skipping it.
  audio = wavfile.read(wav_path)


In [8]:
bits = [2, 4, 6, 8]
wav_path = Path("../data/audio_raw/major.wav")

fig_quantized, ax_quantized = plt.subplots(figsize=(8, 2.2))
fig_error, ax_error = plt.subplots(figsize=(8, 2.2))


first_iteration = True

for b in bits:    
    # Obtener la señal original y la señal cuantizada
    sample_rate, data_cuantizada, data = obtener_cuantizada(wav_path, b)
    wavfile.write(f"../data/audio_cuantizado/major_{b}bits.wav",
                  sample_rate,
                  data_cuantizada.astype(np.int16))
    
    x = np.linspace(0, len(data_cuantizada) / sample_rate, len(data_cuantizada))
    print(f"Valores únicos en la señal cuantizada a {b} bits: {len(np.unique(data_cuantizada))}")
    
    # Datos entre 2 y 2.005 segundos
    idx = (x > 2) & (x < 2.005)
    x_filtered = x[idx]
    data_cuantizada_filtered = data_cuantizada[idx, 1]
    data_filtered = data[idx, 1]
    
    # Imprime la señal original solo en la primera iteración
    if first_iteration:
        ax_quantized.plot(x_filtered, data_filtered, label='Original', 
                         alpha=0.7, color='black', linewidth=1.5)
        first_iteration = False
    
    # Grafica la señal cuantizada para cada profundidad de bits
    ax_quantized.plot(x_filtered, data_cuantizada_filtered, 
                     label=f'Cuant. {b} bits', alpha=0.9)

    # Grafica el error de cuantización para cada profundidad de bits
    ax_error.plot(x_filtered, np.abs(data_filtered - data_cuantizada_filtered),
                 label=f'Error {b} bits')

# Configure and save the quantized signal plot
ax_quantized.set_xlabel('Tiempo [s]')
ax_quantized.set_ylabel('Amplitud')
# legend outside the plot
ax_quantized.legend(loc="upper left", bbox_to_anchor=(1.01, 1))
#ax_quantized.grid()
fig_quantized.savefig('../documento/figure/major_cuantizada_all.pdf', bbox_inches='tight')
plt.close(fig_quantized)

# Configure and save the error plot
ax_error.set_xlabel('Tiempo [s]')
ax_error.set_ylabel('Amplitud')
ax_error.legend(loc="upper left", bbox_to_anchor=(1.01, 1))
#ax_error.grid()
fig_error.savefig('../documento/figure/major_error_cuantizacion_all.pdf', bbox_inches='tight')
plt.close(fig_error)

Valores únicos en la señal cuantizada a 2 bits: 4
Valores únicos en la señal cuantizada a 4 bits: 16
Valores únicos en la señal cuantizada a 6 bits: 64
Valores únicos en la señal cuantizada a 8 bits: 256


In [9]:
bits = [2,4,6,8]
wav_path = Path("../data/audio_raw/Guitar_MoreThanWords_16bit_44.1kHz_stereo.wav")

for b in bits:    
    sample_rate, data_cuantizada, _ = obtener_cuantizada(wav_path, b)
    wavfile.write(f"../data/audio_cuantizado/Guitar_MoreThanWords_16bit_44.1kHz_stereo_{b}bits.wav",
                  sample_rate,
                  data_cuantizada.astype(np.int16))

## Problema 4
